In [ ]:
import pandas as pd
import numpy as np
fn = '../src/data/raw/2005-2015_Greenhouse_Gas_Emissions.csv'

emissions = pd.read_csv(fn)
trunc_emissions = emissions.iloc[5:13]
trunc_emissions.loc[5:13, 'id'] = np.array(np.arange(len(trunc_emissions)))
trunc_emissions

In [ ]:
year_str = [str(int(blah)) for blah in np.linspace(2005, 2015, 11)]
year_str

In [ ]:
# trunc_long = pd.wide_to_long(trunc_emissions, ['20'], i="id", j="year")
# trunc_long.rename(index = int)
# trunc_long
# Remove totals and add leveled indices 
#see https://pandas.pydata.org/pandas-docs/version/0.22/groupby.html#grouping-dataframe-with-index-levels-and-columns
idx_list = np.linspace(5, 13, 9)[:-1]
#non_totals = emissions['Emission Source'][idx_list]
non_totals = trunc_emissions['Emission Source']
type_list = [list(non_totals.values),
             [non_totals.str.split('-')[i][-1].strip().capitalize() for i in idx_list],
             [non_totals.str.split('-')[i][0].strip().capitalize() for i in idx_list]]
idx = pd.MultiIndex.from_arrays(type_list, names=['original', 'source', 'use'])
len_columns = [len(emissions.columns)]
#emissions_lvld = pd.DataFrame(emissions.iloc[idx_list,1:], index = idx)
emissions_lvld = emissions.iloc[idx_list, 1:].copy()
emissions_lvld.index = idx
#emissions.iloc[idx_list,1:]
#emissions_lvld
emissions_lvld.loc[:, 'id'] = np.array(np.arange(len(emissions_lvld)))
emissions_lvld.loc[:, 'source'] = type_list[1]
emissions_lvld.loc[:, 'use'] = type_list[2]

emissions_long = emissions_lvld.melt(col_level=0, id_vars=['id', 'source', 'use'], value_vars=year_str, var_name='year')

#emissions_long = trunc_emissions.melt(col_level=0, id_vars=['id'], value_vars=year_str, var_name='year')
emissions_long['id'] = emissions_long.index
#emissions_long = emissions_long.melt(id_vars = ['id'], value_vars = 'Emission Source')
emissions_long

In [ ]:
len_data = len(emissions_long)
interval = 8
# link_data = np.array([[int(blah) for blah in np.linspace(0,len_data-interval,len_data-interval+1)], 
#                       [int(blah) for blah in np.linspace(interval,len_data,len_data-interval+1)],
#                       emissions_long['value'].values[0:len_data-interval]]).T
link_data = np.transpose([emissions_long['id'].values[0:len_data-interval],
                          emissions_long['id'].values[interval:len_data],
                          emissions_long['value'].values[interval:len_data]])
link_df = pd.DataFrame(link_data, columns = ['source_id','target_id', 'strength'], dtype=object)
link_df

In [ ]:
emissions_long.to_csv('emissions_long.csv')
link_df.to_csv('emissions_link.csv')